### Fluid infiltration and loss

This will ultimately be a key example, because it's very relevant to subduction zones, which are probably the most dramatic context in which geophysics and geochemistry should interplay.

At the beginning of each step we are introducing a fluid phase - i.e. a mobile phase that is able to redistribute chemical components via fractionation and transport. The introduced fluid phase modifies the rbi object prior to the THERMOCALC calculation, i.e. the change of bulk composition caused by the infiltration of fluid causes a change in the phase assemblage. However, normally we would expect that there would still be some sort of fluid present in the new assemblage, probably a silicate melt - and we will then remove that.

At the coarsest scale, we could envisage that the "system" is in the mantle wedge. The infiltrating fluid represents an aqueous fluid arriving in the mantle wedge, having been lost from the top of the downgoing slab as it heats up and dehydrates. The fluid that is lost represents magma that continues upwards towards the volcanic arc.

More generally, this represents a general calc at one point on a spatial grid that needs to have its chemical variables associated with those at other points.

To notice in this file: for now, just that we are adding to the initial rbi construct within the PT_step loop.

I will focus on working up this example.

In [1]:
import tawnycalc as tc

In [2]:
# set up path taken and phases added/removed
removal_fraction = {}
removal_fraction["g"] = 0.8
removal_fraction["bi"] = -0.1

PT_steps = []
PT_steps.append( (11.0, 600 ) )
#PT_steps.append( (12.0, 630 ) )
#PT_steps.append( (12.2, 650 ) )
#PT_steps.append( (11.5, 670 ) )
#PT_steps.append( (10.0, 660 ) )

In [3]:
# To start with a clean context, set `scripts_dir=None`
context = tc.Context(scripts_dir=None)
context.prefs

calcmode   : 1
scriptfile : sukwcu
dataset    : None

In [4]:
# Set the required dataset on the `prefs` dictionary.
context.prefs["dataset"] = 62

In [5]:
context.script[       "axfile"] = "mb50NCKFMASHTO"
context.script[         "with"] = "chl bi pa ru g ilmm sph"  # replaces 'which'
context.script[     "inexcess"] = "mu q H2O"
context.script[       "dogmin"] = "1"                 # replaces 'dogmin yes 1'
context.script[       "maxvar"] = "6"                 # replaces 'setmaxvar'
context.script[    "diagramPT"] = "2 20 400 1100"     # new script
# context.script[        "calcP"] = "11"              # replaces 'setPwindow 11 11'
# context.script[        "calcT"] = "600"             # replaces 'setTwindow 600 600'
context.script["pseudosection"] = ""                  # replaces 'pseudosection yes'
context.script[   "samecoding"] = ["mu pa", "sp mt"]  # 

In [6]:
# Add 'rbi' section.
# We have specific `rbi` object that we use to achieve this. 
# Let's create rbi object, noting how we provide the oxide list at construction time:
oxidestr = "  H2O SiO2 Al2O3 CaO MgO FeO K2O Na2O TiO2 O"
rbi_sec = tc.rbi(oxides=oxidestr)

In [7]:
# Now we need to add the individual phases.
# We will use the `add_phase` method to achieve this. 
# We supply the `phase` name, `mode` data, and finally
# `oxides` as a string or list
#
# Strings and floats can generally be used interchangeably.
#
help(rbi_sec.add_phase)

Help on method add_phase in module tawnycalc.data_objects:

add_phase(phase, mode, oxides) method of tawnycalc.data_objects.rbi instance
    Add a phase to the rbi table. 
    
    Params
    ------
    phase:  str
        Name of phase to add. 
    mode: str, float
        Mode/proportion of phase.
    oxides: str,list
        Proportion of each oxide for phase.



In [8]:
# Construct initial bulk composition

rbi_sec.add_phase(  "g",  "0.02"     , "       0    3.000000    0.984049    0.230147    0.395429    2.406326           0           0           0    0.015951")
rbi_sec.add_phase( "mu",  "0.159799", "1.000000    3.102219    1.393080    0.004483    0.058473    0.053149    0.386538    0.111220           0    0.002460")
rbi_sec.add_phase( "pa",  "0.006915", "       1    2.984010    1.505704    0.019628    0.001993    0.002589    0.033770    0.456416           0    0.000473")
rbi_sec.add_phase( "bi",  "0.090237", "0.932353    2.842048    0.636750           0    1.244448    1.572357    0.500000           0    0.067647    0.021202")
rbi_sec.add_phase("ilmm",  "0.004242", "       0           0           0           0    0.011126    1.032649           0           0    0.956225    0.043775")
rbi_sec.add_phase(  "q",  "0.240992", "       0           1           0           0           0           0           0           0           0           0")
rbi_sec.add_phase("H2O",  "0.458826", "       1           0           0           0           0           0           0           0           0           0")

context.script["rbi"] = rbi_sec   

In [9]:
# Now create xyzguess section.  
# We use the `xyz` object this time. 
# It is really just a Python `OrderedDict`.
# First create a dictionary to record values
xyzguess = tc.xyz()
# Now add values
xyzguess[    "x(g)"] = "0.885508"
xyzguess[    "z(g)"] = "0.211255"
xyzguess[    "f(g)"] = "0.070810"
xyzguess[   "x(bi)"] = "0.579607"
xyzguess[   "y(bi)"] = "0.140313"
xyzguess[   "f(bi)"] = "0.126258"
xyzguess[   "t(bi)"] = "0.072042"
xyzguess[   "Q(bi)"] = "0.121282"
xyzguess[   "x(mu)"] = "0.461998"
xyzguess[   "y(mu)"] = "0.944788"
xyzguess[   "f(mu)"] = "0.009495"
xyzguess[   "n(mu)"] = "0.253509"
xyzguess[   "c(mu)"] = "0.016160"
xyzguess[   "x(pa)"] = "0.461998"
xyzguess[   "y(pa)"] = "0.996868"
xyzguess[   "f(pa)"] = "0.002376"
xyzguess[   "n(pa)"] = "0.955475"
xyzguess[   "c(pa)"] = "0.038175"
xyzguess[   "x(ma)"] = "0.461998"
xyzguess[   "y(ma)"] = "0.964236"
xyzguess[   "f(ma)"] = "0.009701"
xyzguess[   "n(ma)"] = "0.054672"
xyzguess[   "c(ma)"] = "0.944410"
xyzguess[  "x(chl)"] = "0.473331"
xyzguess[  "y(chl)"] = "0.520964"
xyzguess[  "f(chl)"] = "0.248348"
xyzguess["QAl(chl)"] = "0.230540  range -1.000 1.000"
xyzguess[ "Q1(chl)"] = "0.077845  range -1.000 1.000"
xyzguess[ "Q4(chl)"] = "0.092301  range -1.000 1.000"
xyzguess[   "f(ep)"] = "0.264635"
xyzguess[   "Q(ep)"] = "0.250072  range  0.000 0.500"
xyzguess[  "i(ilmm)"] = "0.829808"
xyzguess[  "g(ilmm)"] = "0.017481"
xyzguess[  "Q(ilmm)"] = "0.760053  range -0.990 0.990"
xyzguess[  "x(mt1)"] = "0.762081"
xyzguess[  "Q(mt1)"] = "0.808179"

context.script["xyzguess"] = xyzguess

In [10]:
modes = []
step = 0
for P,T in PT_steps:
    # set required windows
    context.script["calcP"] = "{}".format(P)
    context.script["calcT"] = "{}".format(T)
    
    # add an infiltrating fluid to the rbi
    rbi_current = context.script["rbi"]
    rbi_current.add_phase("fl",  "0.07", "  1    0     0     0     0     0    0.05    0     0    0")
    context.script["rbi"] = rbi_current
    
    # execute
    results = context.execute()
    modes.append((results.modes,step))
    
    # set resultant rbi back on script
    print("Executing for P,T = {},{}".format(P,T))
    context.script["rbi"] = results.rbi

    # remove required phases
    for phase, removal_frac in removal_fraction.items():
        if phase in context.script["rbi"].keys():
            phase_post_solve = context.script["rbi"][phase]["mode"] 
            context.script["rbi"][phase]["mode"] *= (1.-removal_frac)
            print("Phase {} before = {} after = {}".format(phase, phase_post_solve, context.script["rbi"][phase]["mode"] ))
            # get rid of it here too for viz purposes
    
    # record new values to new dict for viz purposes
    final_modes = {}
    for key,val in context.script["rbi"].items():
        final_modes[key] = val["mode"]
    modes.append((final_modes,step))
    step += 1
    


Executing for P,T = 11.0,600
Phase bi before = 0.102174 after = 0.11239140000000002


In [11]:
# Now we have a complete configuration, let's execute. 
# results = context.execute(print_output=True)